<a class="anchor" id="0.1"></a>
# **Table of Contents**
1. [Introduction](#1)
2. [Business Understanding](#2)
3. [Data Understanding](#3)
4. [Data Preperation](#4)
5. [Modeling](#5)
6. [Evaluation](#6)

# **1. Introduction** <a class="anchor" id="1"></a>
[Table of Contents](#0.1)

# **2. Business Understanding** <a class="anchor" id="2"></a>
[Table of Contents](#0.1)

# **3. Data Understanding** <a class="anchor" id="3"></a>
[Table of Contents](#0.1)

# **4. Data Preperation** <a class="anchor" id="4"></a>
[Table of Contents](#0.1)

# **5. Modeling** <a class="anchor" id="5"></a>
[Table of Contents](#0.1)

# **6. Evaluation** <a class="anchor" id="6"></a>
[Table of Contents](#0.1)